In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
import zipfile
import pprint
from pathlib import Path
import pickle

from ipywidgets import interact, interactive, fixed, interact_manual,Button, HBox, VBox
import ipywidgets as widgets
from ipywidgets import TwoByTwoLayout
from ipywidgets import Button, Layout
from termcolor import colored

from nlp_suite import data_preprocessing
#from style_transfer_chatbot import StyleTransferChatbot

In [3]:
def process_inputs(button_instance):
    temp_files_dir = "./dashboard_temp_files"
    global user_name
    user_name = user_name_widget.value
    
    if user_name != "" and uploader_widget.value != {}:
        # prepping an empty temp folder to hold uploaded data
        if os.path.exists(temp_files_dir):
            shutil.rmtree(temp_files_dir)
        os.makedirs(temp_files_dir)
        
        # saving uploaded zip
        uploaded_file_path = os.path.join(temp_files_dir, "saved-output.zip")
        with open(uploaded_file_path, "wb") as f:
            content = (uploader_widget.value[list(uploader_widget.value.keys())[0]])["content"]
            f.write(content)
            
        # unzipping
        with zipfile.ZipFile(uploaded_file_path, 'r') as zip_ref:
            zip_ref.extractall(temp_files_dir)
        os.remove(uploaded_file_path)
        
        # extracting user's messages from text files
        discord_log_paths = [str(path) for path in list(Path(temp_files_dir).rglob('*.txt'))]
        channel_messages, _ = data_preprocessing.process_discord_data(discord_log_paths, 3)
        
        # saving user messages in cache 
        if user_name in channel_messages:
            cached_user_data_dir = os.path.join("cached_user_data", user_name)
            os.makedirs(cached_user_data_dir, exist_ok=True)
            user_messages_path = os.path.join(cached_user_data_dir, "user_messages.p")
            pickle.dump(channel_messages[user_name], open(user_messages_path, "wb"))
            global user_messages
            user_messages = pickle.load(open(user_messages_path, "rb"))
            
        else:
            return None
    else:
        return None
        

In [4]:
# data input
user_name = ""
user_messages = None

print("Please upload a zipped folder containing chat log file(s), and specify a user. \nThen, click the green button to start the data analysis.")
uploader_widget = widgets.FileUpload(description="Upload Chat", accept='.zip', multiple=False)
user_name_widget = widgets.Text(description='User Name:', disabled=False)
process_data_button_widget = widgets.Button(description='Process Chat Data', disabled=False, button_style='success')
process_data_button_widget.on_click(process_inputs)

display(HBox([uploader_widget, user_name_widget]))
display(process_data_button_widget)

Please upload a zipped folder containing chat log file(s), and specify a user. 
Then, click the green button to start the data analysis.


Button(button_style='success', description='Process Chat Data', style=ButtonStyle())

processing
processing


In [14]:
print(user_name)
pprint.pprint(user_messages[:100])

Victoria
['I AM HOSTING A PARTY STARTING TOMMOROW AT 9:00 :D ENJOY URSLEVES IN THE '
 'PARTAE',
 'SEARCH IT UP AND CONVERT',
 ':D WHATS UR TIME',
 "what's your time zone",
 'by the way you should promo yourself by speaking more often or you will '
 'become a ghost',
 'Pls watch this and click thmbs up if you like it',
 'OMG @Small Fan NOOO SMALL FN WHAT HAPPENED ur role is a ghost:',
 'what cool you have malware eh? oh no app data you should redirect the '
 'foloder to oh whatever hmm mal bytes get rid of them os',
 'thats bad do you have mcafee i do',
 'oh mcafee is awesome i have 1 year for free and its super strong but you '
 'need malware bytes is that the only tab you have open? @IM CHIKN',
 'you have chrome or safari',
 'looks like you didnt do a full screenshot so im guessing you have a pc? idk '
 'hmmm',
 'never heard of smart service',
 'if its not responding close the tab',
 'did you just get ot today?',
 'open that malware thing again see if you can do a cleanup its not all 

In [ ]:
############################ CHATBOT ##########################
###############################################################

style_model_dir = "/home/svcl-oowl/brandon/classes/ECE_229/project/chatbot/style_transfer_paraphrase/style_paraphrase/saved_models/{}/checkpoint-9294".format(user)

chatbot = StyleTransferChatbot(style_model_dir)
chatbot_tokenizer = chatbot.tokenizer

# messages seperator based on https://github.com/huggingface/transformers/issues/9365
seperator = "    "
all_conversation_text = ""
max_position_embeddings = 128

out = widgets.Output(layout={'border': '2px solid black', "height":"400px", "width":"900px","overflow":'scroll'})

@out.capture()
def submit_to_chatbot(next_user_input=""):
    global all_conversation_text
    
    # adding user input to conversation text
    print(colored('> User', 'blue', attrs=["bold"]) + ": {}".format(next_user_input))
    if all_conversation_text != "": 
        next_user_input = seperator + next_user_input
    all_conversation_text += next_user_input
    
    # truncate token length if needed
    curr_token_length = chatbot_tokenizer([all_conversation_text], return_tensors='pt')['input_ids'].shape[1]
    while curr_token_length >= max_position_embeddings:
        all_conversation_text = seperator.join(all_conversation_text.split(seperator)[1:])
        curr_token_length = chatbot_tokenizer([all_conversation_text], return_tensors='pt')['input_ids'].shape[1]
    
    # get response
    bot_stylized_response, bot_response = chatbot.get_response(all_conversation_text)
    print(colored('> Bot (original)', 'green', attrs=["bold"]) + ": {}".format(bot_response))
    print(colored('> Bot (stylized)', 'red', attrs=["bold"]) + ": {}".format(bot_stylized_response))
    print("")
    all_conversation_text += seperator + bot_stylized_response  

def restart(button_instance):
    global all_conversation_text
    all_conversation_text = ""
    out.clear_output()

button = widgets.Button(description='Restart Chat',disabled=False,button_style='')
button.on_click(restart)

text_box = widgets.Text(description="User Chat:")
text_input = interactive(submit_to_chatbot,{'manual': True, "manual_name":"Submit Message"}, next_user_input=text_box)
left = VBox([out,button])
display(HBox([text_input,left]))